In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

import pudl
import pudl.extract.eia_bulk_elec as ebulk
import pudl.transform.eia_bulk_elec as tbulk

In [5]:
fpath = Path('/home/tpb/catalyst/workspace/data/eia_bulk_data/ELEC.zip')
assert fpath.exists()

In [6]:
%%time
raw = ebulk.extract(fpath)

CPU times: user 1min 28s, sys: 747 ms, total: 1min 29s
Wall time: 1min 29s


In [34]:
raw_meta = raw['metadata']
raw_ts = raw['timeseries']

In [127]:
raw_meta.head(2)

,series_id,name,units,f,copyright,source,iso3166,lat,lon,geography,start,end,last_updated,latlon,description,geoset_id,category_id,parent_category_id,notes,childseries
0,ELEC.COST_BTU.COW-PCC-99.M,Average cost of fossil fuels for electricity generation (per Btu) : coal : Pacific Contiguous (total) : all sectors : monthly,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",NaN,NaN,NaN,USA-CA+USA-OR+USA-WA,200801,202203,2022-05-24T10:42:22-04:00,NaN,"Summation of all types of coal; All sectors; Summation of the states in the Pacific Contiguous census region including California, Oregon, and Washington;",ELEC.COST_BTU.COW-99.M,NaN,NaN,NaN,NaN
1,ELEC.COST_BTU.COW-PCN-99.M,Average cost of fossil fuels for electricity generation (per Btu) : coal : Pacific Noncontiguous (total) : all sectors : monthly,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",NaN,NaN,NaN,USA-AK+USA-HI,200801,202203,2022-05-24T10:42:22-04:00,NaN,Summation of all types of coal; All sectors; Summation of the states in the Pacific Noncontiguous census region including Alaska and Hawaii.;,ELEC.COST_BTU.COW-99.M,NaN,NaN,NaN,NaN


In [133]:
tbulk._transform_metadata(raw_meta).head(2)

,series_id,units,last_updated,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
0,ELEC.COST_BTU.COW-PCC-99.M,dollars per million Btu,2022-05-24 10:42:22-04:00,Average cost of fossil fuels for electricity generation (per Btu),coal,Pacific Contiguous (total),all sectors,monthly,COST_BTU,COW,PCC,99,M
1,ELEC.COST_BTU.COW-PCN-99.M,dollars per million Btu,2022-05-24 10:42:22-04:00,Average cost of fossil fuels for electricity generation (per Btu),coal,Pacific Noncontiguous (total),all sectors,monthly,COST_BTU,COW,PCN,99,M


In [132]:
pd.to_datetime(raw_meta['last_updated'])

0       2022-05-24 10:42:22-04:00
1       2022-05-24 10:42:22-04:00
2       2022-05-24 10:42:22-04:00
3       2022-05-24 10:42:22-04:00
4       2022-05-24 10:42:22-04:00
                  ...            
8763    2015-09-23 17:16:32-04:00
8764    2015-08-25 15:52:58-04:00
8765    2015-08-25 15:52:58-04:00
8766    2015-08-25 15:52:58-04:00
8767    2015-08-25 15:52:58-04:00
Name: last_updated, Length: 8768, dtype: object

In [40]:
meta = pd.concat([raw_meta.drop(columns=['name']),
                  tbulk._extract_keys_from_name(raw_meta),
                  tbulk._extract_keys_from_series_id(raw_meta)],
                 axis=1
                )

In [42]:
meta.sample(5)

,series_id,units,f,copyright,source,iso3166,lat,lon,geography,start,end,last_updated,latlon,description,geoset_id,category_id,parent_category_id,notes,childseries,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
4341,ELEC.RECEIPTS_BTU.NG-PA-2.Q,billion Btu,Q,None,"EIA, U.S. Energy Information Administration",USA-PA,NaN,NaN,USA-PA,2008Q1,2022Q1,2022-05-24T10:42:22-04:00,NaN,Natural Gas; Power plants owned by regulated electric utilties that produce electricity only;,ELEC.RECEIPTS_BTU.NG-2.Q,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),natural gas,Pennsylvania,electric utility non-cogen,quarterly,RECEIPTS_BTU,NG,PA,2,Q
599,ELEC.COST_BTU.SUB-LA-1.M,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",USA-LA,NaN,NaN,USA-LA,200801,202203,2022-05-24T10:42:22-04:00,NaN,Subbituminous Coal; Power plants owned by regulated electric utilties;,ELEC.COST_BTU.SUB-1.M,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity generation (per Btu),subbituminous coal,Louisiana,electric utility,monthly,COST_BTU,SUB,LA,1,M
5924,ELEC.RECEIPTS_BTU.NG-US-99.A,billion Btu,A,None,"EIA, U.S. Energy Information Administration",USA,NaN,NaN,USA,2008,2021,2022-02-25T15:25:17-05:00,NaN,Natural Gas; All sectors;,ELEC.RECEIPTS_BTU.NG-99.A,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),natural gas,United States,all sectors,annual,RECEIPTS_BTU,NG,US,99,A
5224,ELEC.RECEIPTS_BTU.SUB-FL-1.Q,billion Btu,Q,None,"EIA, U.S. Energy Information Administration",USA-FL,NaN,NaN,USA-FL,2008Q3,2022Q1,2022-05-24T10:42:22-04:00,NaN,Subbituminous Coal; Power plants owned by regulated electric utilties;,ELEC.RECEIPTS_BTU.SUB-1.Q,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),subbituminous coal,Florida,electric utility,quarterly,RECEIPTS_BTU,SUB,FL,1,Q
6738,ELEC.RECEIPTS_BTU.LIG-MT-1.A,billion Btu,A,None,"EIA, U.S. Energy Information Administration",USA-MT,NaN,NaN,USA-MT,2008,2021,2022-02-25T15:25:17-05:00,NaN,Lignite Coal; Power plants owned by regulated electric utilties;,ELEC.RECEIPTS_BTU.LIG-1.A,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),lignite coal,Montana,electric utility,annual,RECEIPTS_BTU,LIG,MT,1,A


In [43]:
meta.shape, raw_ts.shape

((8768, 29), (692317, 3))

In [44]:
meta.sample(5)

,series_id,units,f,copyright,source,iso3166,lat,lon,geography,start,end,last_updated,latlon,description,geoset_id,category_id,parent_category_id,notes,childseries,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
1126,ELEC.COST_BTU.PEL-CO-98.M,dollars per million Btu,M,None,"EIA, U.S. Energy Information Administration",USA-CO,NaN,NaN,USA-CO,200801,202203,2022-05-24T10:42:22-04:00,NaN,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by companies whose primary purpose is to produce power;",ELEC.COST_BTU.PEL-98.M,NaN,NaN,NaN,NaN,Average cost of fossil fuels for electricity generation (per Btu),petroleum liquids,Colorado,electric power (total),monthly,COST_BTU,PEL,CO,98,M
4156,ELEC.RECEIPTS_BTU.NG-ID-2.M,billion Btu,M,None,"EIA, U.S. Energy Information Administration",USA-ID,NaN,NaN,USA-ID,200801,202203,2022-05-24T10:42:22-04:00,NaN,Natural Gas; Power plants owned by regulated electric utilties that produce electricity only;,ELEC.RECEIPTS_BTU.NG-2.M,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),natural gas,Idaho,electric utility non-cogen,monthly,RECEIPTS_BTU,NG,ID,2,M
4872,ELEC.RECEIPTS_BTU.SUB-NE-1.M,billion Btu,M,None,"EIA, U.S. Energy Information Administration",USA-NE,NaN,NaN,USA-NE,200801,202203,2022-05-24T10:42:22-04:00,NaN,Subbituminous Coal; Power plants owned by regulated electric utilties;,ELEC.RECEIPTS_BTU.SUB-1.M,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),subbituminous coal,Nebraska,electric utility,monthly,RECEIPTS_BTU,SUB,NE,1,M
3925,ELEC.RECEIPTS_BTU.NG-ND-98.M,billion Btu,M,None,"EIA, U.S. Energy Information Administration",USA-ND,NaN,NaN,USA-ND,200801,202203,2022-05-24T10:42:22-04:00,NaN,Natural Gas; Power plants owned by companies whose primary purpose is to produce power;,ELEC.RECEIPTS_BTU.NG-98.M,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),natural gas,North Dakota,electric power (total),monthly,RECEIPTS_BTU,NG,ND,98,M
4569,ELEC.RECEIPTS_BTU.NG-UT-97.M,billion Btu,M,None,"EIA, U.S. Energy Information Administration",USA-UT,NaN,NaN,USA-UT,200801,202203,2022-05-24T10:42:22-04:00,NaN,Natural Gas; Power plants in the industrial sector;,ELEC.RECEIPTS_BTU.NG-97.M,NaN,NaN,NaN,NaN,Receipts of fossil fuels by electricity plants (Btu),natural gas,Utah,all industrial (total),monthly,RECEIPTS_BTU,NG,UT,97,M


In [45]:
ts.head(5)

,series_id,date,value
0,ELEC.COST_BTU.COW-PCC-99.M,2022-03-01,NaN
1,ELEC.COST_BTU.COW-PCC-99.M,2022-02-01,NaN
2,ELEC.COST_BTU.COW-PCC-99.M,2022-01-01,NaN
3,ELEC.COST_BTU.COW-PCC-99.M,2021-12-01,NaN
4,ELEC.COST_BTU.COW-PCC-99.M,2021-11-01,NaN


In [46]:
ts.isna().mean()

series_id    0.000000
date         0.000000
value        0.142286
dtype: float64

In [47]:
ts.isna().groupby(ts['series_id'].str[-1])['value'].mean()

series_id
A    0.191406
M    0.139893
Q    0.137875
Name: value, dtype: float64

In [48]:
ts.memory_usage(deep=True) / 2**20

Index        0.000122
series_id    2.278270
date         5.281960
value        5.281960
dtype: float64

In [49]:
meta.describe()

,lat,lon,category_id,parent_category_id
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


In [50]:
meta.loc[:, ~meta.isna().all()].describe()

,series_id,units,f,copyright,source,iso3166,geography,start,end,last_updated,description,geoset_id,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
count,8768,8768,8768,8768,8768,6837,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768,8768
unique,8768,2,3,1,1,52,62,37,99,23,594,453,2,7,62,12,3,2,7,62,12,3
top,ELEC.COST_BTU.COW-PCC-99.M,billion Btu,M,None,"EIA, U.S. Energy Information Administration",USA,USA,200801,202203,2022-05-24T10:42:22-04:00,Natural Gas; Power plants owned by companies whose primary purpose is to produce power;,ELEC.RECEIPTS_BTU.PEL-98.M,Receipts of fossil fuels by electricity plants (Btu),natural gas,United States,electric utility,monthly,RECEIPTS_BTU,NG,US,1,M
freq,1,6482,3035,8768,8768,450,450,2949,2798,5453,299,61,6482,2256,450,1637,3035,6482,2256,450,1637,3035


In [51]:
all_nan = meta.isna().all()
all_none = meta.eq('None').all()
to_drop = all_nan | all_none
dropped_col_names = meta.columns[to_drop]

In [52]:
set(dropped_col_names)

{'category_id',
 'childseries',
 'copyright',
 'lat',
 'latlon',
 'lon',
 'notes',
 'parent_category_id'}

In [53]:
tbulk._get_empty_col_names(meta)

{'category_id',
 'childseries',
 'copyright',
 'lat',
 'latlon',
 'lon',
 'notes',
 'parent_category_id'}

In [54]:
tbulk._get_redundant_frequency_col_names(meta)

{'f'}

In [55]:
tbulk._get_constant_col_names(meta)

{'copyright', 'source'}

In [56]:
meta['fuel_code'].value_counts()

NG     2256
PEL    1811
COW    1741
BIT    1149
SUB     963
PC      637
LIG     211
Name: fuel_code, dtype: int64

In [57]:
meta.columns[meta.nunique() == 1]

Index(['copyright', 'source'], dtype='object')

In [58]:
geo_parts = meta['geoset_id'].str.split('-', expand=True)

In [59]:
tbulk._get_col_names_to_drop(meta)

{'category_id',
 'childseries',
 'copyright',
 'f',
 'geoset_id',
 'lat',
 'latlon',
 'lon',
 'notes',
 'parent_category_id',
 'source'}

In [60]:
pd.set_option('display.max_colwidth', None)

In [69]:
meta.sample(5, random_state=2)[['series_id', 'description']]

,series_id,description
275,ELEC.COST_BTU.PC-MI-1.M,Petroleum Coke (solid residual petroluem); Power plants owned by regulated electric utilties;
7717,ELEC.COST_BTU.PEL-KY-1.A,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by regulated electric utilties;"
8069,ELEC.COST_BTU.PEL-PCN-1.A,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by regulated electric utilties; Summation of the states in the Pacific Noncontiguous census region including Alaska and Hawaii.;"
4344,ELEC.RECEIPTS_BTU.NG-PCC-99.Q,"Natural Gas; All sectors; Summation of the states in the Pacific Contiguous census region including California, Oregon, and Washington;"
8561,ELEC.RECEIPTS_BTU.COW-CT-3.M,Summation of all types of coal; Power plants owned by regualted electric utilities that produce both electricity and useful heat;


In [63]:
dmeta = meta.drop(columns=tbulk._get_col_names_to_drop(meta))

In [73]:
dmeta.sample(5, random_state=3)

,series_id,units,iso3166,geography,start,end,last_updated,description,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
5460,ELEC.RECEIPTS_BTU.SUB-WI-7.A,billion Btu,USA-WI,USA-WI,2008,2021,2022-02-25T15:25:17-05:00,Subbituminous Coal; Power plants in the industrial sector that produce electricity and useful heat;,Receipts of fossil fuels by electricity plants (Btu),subbituminous coal,Wisconsin,industrial cogen,annual,RECEIPTS_BTU,SUB,WI,7,A
5567,ELEC.RECEIPTS_BTU.PEL-MAT-98.A,billion Btu,NaN,USA-NJ+USA-NY+USA-PA,2008,2021,2022-02-25T15:25:17-05:00,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids); Power plants owned by companies whose primary purpose is to produce power; Summation of the states in the Middle Atlantic census region including New Jersey, New York, and Pennsylvania.;",Receipts of fossil fuels by electricity plants (Btu),petroleum liquids,Middle Atlantic (total),electric power (total),annual,RECEIPTS_BTU,PEL,MAT,98,A
5612,ELEC.RECEIPTS_BTU.SUB-AL-99.A,billion Btu,USA-AL,USA-AL,2008,2021,2022-02-25T15:25:17-05:00,Subbituminous Coal; All sectors;,Receipts of fossil fuels by electricity plants (Btu),subbituminous coal,Alabama,all sectors,annual,RECEIPTS_BTU,SUB,AL,99,A
7686,ELEC.COST_BTU.NG-MI-94.A,dollars per million Btu,USA-MI,USA-MI,2008,2021,2022-02-25T15:25:17-05:00,Natural Gas; Power plants owned by unregulated power companies (also called merchant generators);,Average cost of fossil fuels for electricity generation (per Btu),natural gas,Michigan,independent power producers (total),annual,COST_BTU,NG,MI,94,A
5574,ELEC.RECEIPTS_BTU.SUB-GA-98.A,billion Btu,USA-GA,USA-GA,2008,2021,2022-02-25T15:25:17-05:00,Subbituminous Coal; Power plants owned by companies whose primary purpose is to produce power;,Receipts of fossil fuels by electricity plants (Btu),subbituminous coal,Georgia,electric power (total),annual,RECEIPTS_BTU,SUB,GA,98,A


In [126]:
# confirm connecticut is in "geography" because it is missing from "description"
dmeta.loc[dmeta['region_code'].eq("NEW"),:].head(3)

,series_id,units,iso3166,geography,start,end,last_updated,description,series,fuel,region,sector,frequency,series_code,fuel_code,region_code,sector_code,frequency_code
10,ELEC.COST_BTU.COW-NEW-1.M,dollars per million Btu,NaN,USA-CT+USA-MA+USA-ME+USA-NH+USA-RI+USA-VT,200801,202203,2022-05-24T10:42:22-04:00,"Summation of all types of coal; Power plants owned by regulated electric utilties; Summation of the states in the census region of New England including Maine, Massaschuetts, New Hampshire, Rhode Island and Vermont;",Average cost of fossil fuels for electricity generation (per Btu),coal,New England (total),electric utility,monthly,COST_BTU,COW,NEW,1,M
15,ELEC.COST_BTU.COW-NEW-1.Q,dollars per million Btu,NaN,USA-CT+USA-MA+USA-ME+USA-NH+USA-RI+USA-VT,2008Q1,2022Q1,2022-05-24T10:42:22-04:00,"Summation of all types of coal; Power plants owned by regulated electric utilties; Summation of the states in the census region of New England including Maine, Massaschuetts, New Hampshire, Rhode Island and Vermont;",Average cost of fossil fuels for electricity generation (per Btu),coal,New England (total),electric utility,quarterly,COST_BTU,COW,NEW,1,Q
29,ELEC.COST_BTU.COW-NEW-99.M,dollars per million Btu,NaN,USA-CT+USA-MA+USA-ME+USA-NH+USA-RI+USA-VT,200801,202203,2022-05-24T10:42:22-04:00,"Summation of all types of coal; All sectors; Summation of the states in the census region of New England including Maine, Massaschuetts, New Hampshire, Rhode Island and Vermont;",Average cost of fossil fuels for electricity generation (per Btu),coal,New England (total),all sectors,monthly,COST_BTU,COW,NEW,99,M


In [106]:
dmeta.groupby(["fuel", "fuel_code"])['description'].first().str.strip('; ').str.split(';', expand=True)[[0]]

,,0
fuel,fuel_code,
bituminous coal,BIT,Bituminous Coal
coal,COW,Summation of all types of coal
lignite coal,LIG,Lignite Coal
natural gas,NG,Natural Gas
petroleum coke,PC,Petroleum Coke (solid residual petroluem)
petroleum liquids,PEL,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids)"
subbituminous coal,SUB,Subbituminous Coal


In [105]:
dmeta.groupby(["sector", "sector_code"])['description'].first().str.strip('; ').str.split(';', expand=True)[[1]]

,,1
sector,sector_code,
all commercial (total),96,Power plants in the commercial sector
all industrial (total),97,Power plants in the industrial sector
all sectors,99,All sectors
commercial cogen,5,Power plants in the commercial sector that produce electricity and useful heat
commercial non-cogen,4,Power plants in the commercial sector that produce electricity only
electric power (total),98,Power plants owned by companies whose primary purpose is to produce power
electric utility,1,Power plants owned by regulated electric utilties
electric utility cogen,3,Power plants owned by regualted electric utilities that produce both electricity and useful heat
electric utility non-cogen,2,Power plants owned by regulated electric utilties that produce electricity only


In [107]:
from io import StringIO
pd.read_csv(StringIO(
	"""
fuel	fuel_code	description
bituminous coal	BIT	Bituminous Coal
coal	COW	Summation of all types of coal
lignite coal	LIG	Lignite Coal
natural gas	NG	Natural Gas
petroleum coke	PC	Petroleum Coke (solid residual petroluem)
petroleum liquids	PEL	Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids)
subbituminous coal	SUB	Subbituminous Coal
    """
), sep='\t')

,fuel,fuel_code,description
0,bituminous coal,BIT,Bituminous Coal
1,coal,COW,Summation of all types of coal
2,lignite coal,LIG,Lignite Coal
3,natural gas,NG,Natural Gas
4,petroleum coke,PC,Petroleum Coke (solid residual petroluem)
5,petroleum liquids,PEL,"Summation of all petroleum liquids (distallte fuel oil, jet fuel, residual fuel oil, kerosense waste oil and other petroleum liquids)"
6,subbituminous coal,SUB,Subbituminous Coal


In [115]:
out = StringIO()
dmeta.groupby(["region", "region_code"])['description'].first().str.strip('; ').str.split(';', expand=True)[[2]].to_csv(out, sep=",")

In [125]:
print(out.getvalue())

region	region_code	2
Alabama	AL	
Alaska	AK	
Arizona	AZ	
Arkansas	AR	
California	CA	
Colorado	CO	
Connecticut	CT	
Delaware	DE	
District Of Columbia	DC	
East North Central (total)	ENC	 Summation of the states in the East North Central census region including Illinois, Indiana, Michigan, Ohio and Wisconsin.
East South Central (total)	ESC	 Summation of the states in the East South Central census region including Alabama, Kentucky, Mississippi, and Tennessee
Florida	FL	
Georgia	GA	
Hawaii	HI	
Idaho	ID	
Illinois	IL	
Indiana	IN	
Iowa	IA	
Kansas	KS	
Kentucky	KY	
Louisiana	LA	
Maine	ME	
Maryland	MD	
Massachusetts	MA	
Michigan	MI	
Middle Atlantic (total)	MAT	 Summation of the states in the Middle Atlantic census region including New Jersey, New York, and Pennsylvania.
Minnesota	MN	
Mississippi	MS	
Missouri	MO	
Montana	MT	
Mountain (total)	MTN	 Summation of the states in the Mountain census region including Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah and Wyoming.
Nebraska	NE	
Neva

[autoreload of pudl.transform.eia_bulk_elec failed: Traceback (most recent call last):
  File "/home/tpb/miniconda3/envs/pudl-dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/tpb/miniconda3/envs/pudl-dev/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/tpb/miniconda3/envs/pudl-dev/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/tpb/catalyst/pudl/src/pudl/transform/eia_bulk_elec.py", line 164, in <module>
    EIA_FUELS = pd.read_csv(
  File "/home/tpb/miniconda3/envs/pudl-dev/lib/python3.10/site-packages/pandas/util/_decorators.py", line 311, i

In [134]:
wide_ts = ts.merge(dmeta[['series_id', 'units', 'fuel', 'region','sector', 'frequency']], on='series_id').drop(columns='series_id')

In [136]:
wide_ts = wide_ts.set_index(['units', 'fuel', 'region','sector', 'frequency', 'date']).unstack('units')

In [137]:
wide_ts.columns = wide_ts.columns.droplevel(level=None)

In [139]:
wide_ts.rename(columns={'billion Btu': 'receipts_billion_btu',
                       'dollars per million Btu': 'cost_dollars_per_mmbtu'}, inplace=True)

In [140]:
wide_ts.sample(5)

units                                                                                  receipts_billion_btu  cost_dollars_per_mmbtu
fuel              region                  sector                 frequency date                                                    
bituminous coal   Wisconsin               all sectors            monthly   2013-11-01            5093.44289                     NaN
natural gas       Middle Atlantic (total) electric utility       monthly   2019-05-01            5584.22413                  2.5061
petroleum liquids Arizona                 all sectors            quarterly 2019-07-01              86.67529                     NaN
                  Georgia                 electric power (total) monthly   2017-11-01              65.23411                 13.4328
natural gas       North Dakota            industrial cogen       monthly   2020-01-01               0.00000                     NaN

In [142]:
wide_ts.columns.name = None

In [143]:
wide_ts.columns

Index(['receipts_billion_btu', 'cost_dollars_per_mmbtu'], dtype='object')

In [144]:
tbulk._transform_timeseries(ts, dmeta).sample(5)

receipts_billion_btu  cost_dollars_per_mmbtu
fuel              region                     sector                 frequency date                                                    
petroleum liquids Minnesota                  electric utility cogen monthly   2019-12-01               0.00000                     NaN
                  Hawaii                     electric utility       annual    2017-01-01           43336.50406                     NaN
natural gas       South Dakota               electric power (total) quarterly 2019-04-01            2147.10465                2.170211
coal              Pennsylvania               industrial cogen       monthly   2019-09-01             285.49144                     NaN
petroleum coke    East South Central (total) all sectors            monthly   2008-03-01            3140.56219                     NaN

In [148]:
cen_reg = pd.read_csv(
    StringIO(
        """
region,region_code,included_states
East North Central,ENC,"Illinois,Indiana,Michigan,Ohio,Wisconsin"
East South Central,ESC,"Alabama,Kentucky,Mississippi,Tennessee"
Middle Atlantic,MAT,"New Jersey,New York,Pennsylvania"
Mountain,MTN,"Arizona,Colorado,Idaho,Montana,Nevada,New Mexico,Utah,Wyoming"
New England,NEW,"Maine,Massaschuetts,New Hampshire,Rhode Island,Vermont,Connecticut"
Pacific Contiguous,PCC,"California,Oregon,Washington"
Pacific Noncontiguous,PCN,"Alaska,Hawaii"
South Atlantic,SAT,"Delaware,the District of Columbia,Florida,Georgia,Maryland,North Carolina,South Carolina,Virginia,West Virginia"
West North Central,WNC,"Iowa,Kansas,Minnesota,Missouri,Nebraska,North Dakota,South Dakota"
West South Central,WSC,"Arkansas,Louisiana,Oklahoma,Texas"
    """
    ),
)

In [149]:
cen_reg

,region,region_code,included_states
0,East North Central,ENC,"Illinois,Indiana,Michigan,Ohio,Wisconsin"
1,East South Central,ESC,"Alabama,Kentucky,Mississippi,Tennessee"
2,Middle Atlantic,MAT,"New Jersey,New York,Pennsylvania"
3,Mountain,MTN,"Arizona,Colorado,Idaho,Montana,Nevada,New Mexico,Utah,Wyoming"
4,New England,NEW,"Maine,Massaschuetts,New Hampshire,Rhode Island,Vermont,Connecticut"
5,Pacific Contiguous,PCC,"California,Oregon,Washington"
6,Pacific Noncontiguous,PCN,"Alaska,Hawaii"
7,South Atlantic,SAT,"Delaware,the District of Columbia,Florida,Georgia,Maryland,North Carolina,South Carolina,Virginia,West Virginia"
8,West North Central,WNC,"Iowa,Kansas,Minnesota,Missouri,Nebraska,North Dakota,South Dakota"
9,West South Central,WSC,"Arkansas,Louisiana,Oklahoma,Texas"


In [152]:
cen_reg['included_states'].str.split(',', expand=True).melt()

,variable,value
0,0,Illinois
1,0,Alabama
2,0,New Jersey
3,0,Arizona
4,0,Maine
...,...,...
85,8,None
86,8,None
87,8,West Virginia
88,8,None


In [160]:
pd.concat([cen_reg[['region', 'region_code']], cen_reg['included_states'].str.split(',', expand=True)], axis=1).melt(id_vars=['region', 'region_code'], value_name='state').dropna(subset='state').drop(columns='variable').sort_values('region')

,region,region_code,state
0,East North Central,ENC,Illinois
20,East North Central,ENC,Michigan
40,East North Central,ENC,Wisconsin
10,East North Central,ENC,Indiana
30,East North Central,ENC,Ohio
21,East South Central,ESC,Mississippi
1,East South Central,ESC,Alabama
31,East South Central,ESC,Tennessee
11,East South Central,ESC,Kentucky
12,Middle Atlantic,MAT,New York
